In [1]:
import base64

In [2]:
import requests
import datetime
from urllib.parse import urlencode

In [3]:
!pip install requests

In [4]:
client_id = 'dd67a453d722441da636647587d4a7be'
client_secret = '26cf26febfd24315aa047ef52566d669'


In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [6]:

spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.perform_auth()

True

In [8]:
access_token = spotify.access_token
access_token

'BQAzlu2E3wnwzSX-mveIEeVmbFrO8B8mM1u8XD94DD9DwyMiiTiHsXUYdoF1YGlWtPA4lR0KIzp6kRNz9eM'

In [13]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q": "JEREMY SHIELDS", "type": "artist"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=JEREMY+SHIELDS&type=artist
https://api.spotify.com/v1/search?q=JEREMY+SHIELDS&type=artist
200


In [14]:
r.json()

{'artists': {'href': 'https://api.spotify.com/v1/search?query=JEREMY+SHIELDS&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3xHvrNjNO8GFy3kq4tSdOb'},
    'followers': {'href': None, 'total': 8},
    'genres': [],
    'href': 'https://api.spotify.com/v1/artists/3xHvrNjNO8GFy3kq4tSdOb',
    'id': '3xHvrNjNO8GFy3kq4tSdOb',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/38a2398a71498fca9f47c88b8a84f3bf331a8c70',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/96492d31f3491df549a4d37b0e3f0345d7d2f996',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/f78a46e06fc6e7aa7875f6d54d61ec1082365561',
      'width': 160}],
    'name': 'Jeremy Shields',
    'popularity': 0,
    'type': 'artist',
    'uri': 'spotify:artist:3xHvrNjNO8GFy3kq4tSdOb'}],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 1}}